In [2]:
import nltk
import string
from nltk.util import ngrams
import math
import numpy as np

# Assistance functions

In [3]:
# to find n Choose r
def nCr(n,r):
    r2 = max(r,n-r)
    ncr = 1
    for m in range(n,r2,-1):
        ncr = ncr*m
    for m in range(1,n-r2+1):
        ncr = ncr//m
    return ncr


# split sentence
def sent_splitter(sent, add_tag=1):
    if type(sent)==list or type(sent)==tuple:
        return sent
    elif type(sent)==str:
        punctuation_char = string.punctuation + '“”'
        sent = sent.translate(str.maketrans('','',punctuation_char))
        if add_tag:
            sent = '<s> '+ sent.lower()+ ' </s>'
        sent = sent.split(sep=' ')
        return sent
    else:
        print(sent)
        print("Invalid data passed to sentence splitter")
        return -1

# Preprocessing of Corpus

In [4]:
f = open('theAdventuresofSherlockHolmes.txt',encoding="utf8")
# f = open('AliceAdventuresinWonderland.txt',encoding="utf8")

corpus = f.read()
corpus = corpus.replace("'", ' ')
corpus = corpus.replace("-", ' ')
# Removing unneccessay punctuation characters except .,?,!
punctuation_char = string.punctuation + '“”'
punctuation_char = punctuation_char.replace('!','')
punctuation_char = punctuation_char.replace('?','')
punctuation_char = punctuation_char.replace('.','')
corpus = corpus.translate(str.maketrans('','',punctuation_char))
corpus = corpus.replace('--', '')
# Some unneccessary Roman Numericals
corpus = corpus.replace('\nI.', '')
corpus = corpus.replace('\nII.', '')
corpus = corpus.replace('\nIII.', '')
# Removing new line characters
corpus = corpus.replace('\n', ' ')

# Splitting Corpus into Sentences
sent_corpus = nltk.tokenize.sent_tokenize(corpus)

# Adding start and end of sentence tags
for i in range(len(sent_corpus)):
    sent_corpus[i] = '<s> '+ sent_corpus[i].replace('.','').lower()+ ' </s>'     

# Splitting Sentences into words
words_sent_corpus = []
# count_words = 0
for i in range(len(sent_corpus)):
    words_sent_corpus.append(sent_corpus[i].split(sep=' '))
#     count_words += len(sent_corpus[i].split(sep=' '))-2

## Spliting Corpus 80:20

In [5]:
l = len(words_sent_corpus)
words_sent_corpus_testing = words_sent_corpus[math.ceil(0.8*l):]
words_sent_corpus = words_sent_corpus[:math.ceil(0.8*l)]

# Finding n-grams and their counts

In [6]:
def find_ngram(n):
    list_ngram = []
    for w in words_sent_corpus:
        for w2 in list(ngrams(w,n)):
            if n==1:
                list_ngram.append((w2))
            else:
                if w2!=('','','','') and w2!=('','','') and w2!=('','') and w2!=(''):
                    list_ngram.append(w2)
    return list_ngram

dict_ngram={} # key= n, value= ngram
for i in range(1,5):
    dict_ngram.update({i:find_ngram(i)})

# Count of k word sequence's for k = 1,2,3,4 for kth order MLE
# 1-unigram,2-bigram,3-trigram,4-quadgram
dict_wordseq_count = {1:{},2:{},3:{},4:{}}  
def form_count_for_MLE(n):
    global dict_wordseq_count
    if len(dict_wordseq_count[n])!=0: # to ensure this function isn't executed twice in the notebook
        pass
    else:
        for i in dict_ngram[n]:
            try:
                dict_wordseq_count[n][i]+=1
            except KeyError:
                dict_wordseq_count[n].update({i:1})
for i in range(1,5):
    form_count_for_MLE(i)

# Number of ngrams present and number of ngrams possible:

In [7]:
for i in range(1,5):
    print("Number of {}-gram present (counting repetitions): {}".format(str(i),len(dict_ngram[i])))
    print("Number of Unique {}-gram: {}".format(str(i),len(dict_wordseq_count[i])))
    print("Number of {}-gram possible: {}\n".format(str(i),nCr(len(dict_wordseq_count[i]),i)))

Number of 1-gram present (counting repetitions): 94997
Number of Unique 1-gram: 7598
Number of 1-gram possible: 7598

Number of 2-gram present (counting repetitions): 89368
Number of Unique 2-gram: 42124
Number of 2-gram possible: 887194626

Number of 3-gram present (counting repetitions): 83961
Number of Unique 3-gram: 69349
Number of 3-gram possible: 55584099100474

Number of 4-gram present (counting repetitions): 78540
Number of Unique 4-gram: 75112
Number of 4-gram possible: 1326146093198017070



# Maximum likelihood Estimate for ngram

###  Function to find probabity of given word sequence

In [8]:
def mle_p_ngram(n,wlist):
#     if type(wlist)==str:
    wlist = sent_splitter(wlist,add_tag=0)
        
    if len(wlist)!=n:
        print("Invalid model, number of words is not equal to {}".format(n))
        return -1
    try:  
        if n==1:
            return float(dict_wordseq_count[n][(wlist[0],)])/(len(dict_ngram[n]))
        elif n==2:
            return float(dict_wordseq_count[n][tuple(wlist)])/dict_wordseq_count[n-1][(wlist[0],)]
        elif n==3:
            return float(dict_wordseq_count[n][tuple(wlist)])/dict_wordseq_count[n-1][wlist[0],wlist[1]]
        elif n==4:
            return float(dict_wordseq_count[n][tuple(wlist)])/dict_wordseq_count[n-1][wlist[0],wlist[1],wlist[2]]
    except KeyError: #Word not found
        return 0

n=2
w = 'with an'
# mle_p_ngram(n,w)

print("Example Sentences with Probability:")
sl = ['visit','i sat','do you mean','i had got when']
for i in range(4):
    print('\nSentence: {}\n'.format(sl[i]))
    try:
        print("{}-Gram Model, Probability:{}".format(i+1,mle_p_ngram(i+1,sl[i])))
    except:
        pass

Example Sentences with Probability:

Sentence: visit

1-Gram Model, Probability:9.473983388949125e-05

Sentence: i sat

2-Gram Model, Probability:0.0022271714922048997

Sentence: do you mean

3-Gram Model, Probability:0.045454545454545456

Sentence: i had got when

4-Gram Model, Probability:1.0


### Find probability for all ngrams present in corpus

In [9]:
mle_word_dict = {1:{},2:{},3:{},4:{}}
for n in range(1,5):
    for w in dict_wordseq_count[n]:
            try:
                mle_word_dict[n][w] += mle_p_ngram(n,w)
            except KeyError:
                mle_word_dict[n].update({w:mle_p_ngram(n,w)})
                    
mle_wordseq_prob_list = {1:[],2:[],3:[],4:[]}
mle_wordseq_list = {1:[],2:[],3:[],4:[]}

for n in range(1,5):
    for w in mle_word_dict[n]:
        mle_wordseq_prob_list[n].append(mle_word_dict[n][w])
        mle_wordseq_list[n].append(w)

# Function to find probability of sentence for given ngram model

In [10]:
def probability_sentence(sent,model_no):
    sent=sent_splitter(sent,add_tag=1)
    prob = 0
    if model_no==1:
        l = -len(sent)
    else:
        l = model_no
    for w in sent[:-l+1]:
        index = sent.index(w)
        if index+model_no<=len(sent):
            temp = mle_p_ngram(model_no,sent[index:index+model_no])
#             print(temp)
            if temp==0: # How to take log
                probt = float("-inf")
            else:
                probt = math.log(temp)
            prob += probt
#             print (prob)
    return math.exp(prob)


print("Example Sentences with Probability:\n")
sl = ['i sat down','do you mean that','the lamps had been lit but the blinds had not been drawn','so far i had got when we went to visit the scene of action']
for s in sl:
    print('\nSentence: {}\n'.format(s))
    for i in range(1,4):
        print("{}-Gram Model, Probability:{}".format(i,probability_sentence(s,i)))

Example Sentences with Probability:


Sentence: i sat down

1-Gram Model, Probability:6.501378758699384e-11
2-Gram Model, Probability:2.9039371565473137e-06
3-Gram Model, Probability:7.450176941702365e-05

Sentence: do you mean that

1-Gram Model, Probability:1.4668322521718687e-13
2-Gram Model, Probability:9.5934619118052e-09
3-Gram Model, Probability:0.0

Sentence: the lamps had been lit but the blinds had not been drawn

1-Gram Model, Probability:2.524209073501141e-36
2-Gram Model, Probability:0.0
3-Gram Model, Probability:2.0156754402622197e-10

Sentence: so far i had got when we went to visit the scene of action

1-Gram Model, Probability:4.4761176669234027e-41
2-Gram Model, Probability:1.0186416757951622e-25
3-Gram Model, Probability:4.16666666666667e-08


# Sentence Generator

In [11]:
def prob_ngrams_starting_with_w(w_seq, model_no):
    prob_list = []
    word_list = []
    indexing_list = []
    if type(w_seq)==str:
        w_seq= sent_splitter(w_seq, add_tag=0)
#     print('prob_ngrams_w',w_seq)
    for i in range(len(mle_wordseq_list[model_no])):
        if model_no in [2,3,4]:
            if mle_wordseq_list[model_no][i][:model_no-1] == tuple(w_seq):
                indexing_list.append(len(word_list))
                word_list.append(mle_wordseq_list[model_no][i])
                prob_list.append(mle_wordseq_prob_list[model_no][i])
        elif model_no==1:
            indexing_list.append(len(word_list))
            word_list.append(mle_wordseq_list[model_no][i])
            prob_list.append(mle_wordseq_prob_list[model_no][i])            
    return word_list, prob_list, indexing_list

def sent_generate(model_no):
    w1 = '<s>'
    we = '</s>'
    sfinal = '<s>'
    if model_no==1:
        w_seq = '<s>'
        count = 0
        while w_seq != '</s>':
            word_list,prob_list,indexing_list = prob_ngrams_starting_with_w(w_seq,model_no)
            next_w_index = np.random.choice(indexing_list,1,prob_list)
            w_seq = word_list[int(next_w_index)][0]
            sfinal += ' '+ w_seq
            if count == 25:
                break
            count +=1
        return sfinal
    
    elif model_no==2:
        w_seq = '<s>'
        while w_seq != '</s>':    
            word_list,prob_list,indexing_list = prob_ngrams_starting_with_w((w_seq,),model_no)
            next_w_index = np.random.choice(indexing_list,1,prob_list)
            w_seq = word_list[int(next_w_index)][1]
            sfinal += ' '+ w_seq
        return sfinal
    elif model_no==3:
        w_seq = ['<s>']
        # First get second word from bigram counts. 
        # This is valid as count('<s>','<s>',w)=count('<s>',w) in our case as we are not taking multiple '<s>'
        word_list,prob_list,indexing_list = prob_ngrams_starting_with_w((w_seq[0],),model_no-1)
        next_w_index = np.random.choice(indexing_list,1,prob_list)
        w_seq.append(word_list[int(next_w_index)][1])
#         print(w_seq)
        sfinal += ' '+ word_list[int(next_w_index)][-1]
        while w_seq[1] != '</s>':
            word_list, prob_list, indexing_list = prob_ngrams_starting_with_w(w_seq,model_no)
            next_w_index = np.random.choice(indexing_list,1,prob_list)
            w_seq = word_list[int(next_w_index)][1:]
#             print(w_seq)
            sfinal += ' '+ w_seq[-1]
        return sfinal
    elif model_no==4:
        w_seq = ['<s>']
        # First get second word from bigram counts. 
        # This is valid as count('<s>','<s>',w)=count('<s>',w) in our case as we are not taking multiple '<s>'
        # Similarly get third word from trigram
        word_list,prob_list,indexing_list = prob_ngrams_starting_with_w((w_seq[0],),model_no-2)
        next_w_index = np.random.choice(indexing_list,1,prob_list)
        w_seq.append(word_list[int(next_w_index)][1])
#         print(w_seq)
        sfinal += ' '+ word_list[int(next_w_index)][-1]
        
        word_list, prob_list, indexing_list = prob_ngrams_starting_with_w(w_seq,model_no-1)
        next_w_index = np.random.choice(indexing_list,1,prob_list)
        w_seq = word_list[int(next_w_index)][:]
#         print(w_seq)
        sfinal += ' '+ word_list[int(next_w_index)][-1]
        
        while w_seq[2] != '</s>':
            word_list, prob_list, indexing_list = prob_ngrams_starting_with_w(w_seq,model_no)
            next_w_index = np.random.choice(indexing_list,1,prob_list)
            w_seq = word_list[int(next_w_index)][1:]
#             print(w_seq)
            sfinal += ' '+ w_seq[-1]
        return sfinal
        
print("Example Sentences generated using NGram Model:\n")
for i in range(1,5):
    print('\n{}-Gram Model\n'.format(i))
    print(sent_generate(i))
    print(sent_generate(i))
    print(sent_generate(i))

Example Sentences generated using NGram Model:


1-Gram Model

<s> sholto clock? faded bachelor exceedingly 2000 income turn cobwebby commit track creatures 85 avail epistle troubled discriminate four heads nod don aspired wrist pshaw! grey hum!
<s> mahogany completely emotions brain commenting overhauled jove! curb like varieties commissions expectancies fat reared intrusted glances eligible hide groomed crisp hawk hercules enigmatical nut requested discover
<s> dried wouldn madam? grinned k forgive lure antecedents boasting georgia throat scream fangs wore repeatedly cal trouble! scuffle longed encompass ashen sleeper concisely merit fills dig

2-Gram Model

<s> fritz! </s>
<s> american had come upon those lines at exactly my limits in monosyllables and unclaspings of proof with eager nature rather suddenly and gave a boarding school what mr lestrade rose from off however there may take in bewilderment at anything to affect the continent </s>
<s> three pound a whip across is itself i

## Add-1 smoothing for bigram + Updated count:

In [12]:
def add1_bigram_prob(wlist):
#     if type(wlist)==str:
    wlist=sent_splitter(wlist,add_tag=0)
    if len(wlist)!=2:
        print("Not a Bigram!")
        return -1
    else:
        try:
            return (float(dict_wordseq_count[2][tuple(wlist)])+1)/(dict_wordseq_count[1][(wlist[0],)]+ len(dict_wordseq_count[1]))
        except KeyError:
            try:
                return (1)/(dict_wordseq_count[1][(wlist[0],)]+ len(dict_wordseq_count[1]))
            except KeyError:
#               ("The 1st word of bigram doesnt exist in training corpus".format(wlist[0]))
                return 1/(1+len(dict_wordseq_count[1]))
def add1_bigram_count(wlist):
    if type(wlist)==str:
        wlist=sent_splitter(wlist,add_tag=0)
    if len(wlist)!=2:
        print("Not a Bigram!")
        return -1
    else:
        return dict_wordseq_count[1][(wlist[0],)]*add1_bigram_prob(wlist)
    
print("Examples with Drastic change in Counts:\n")
wl = [('of','the'),('it', 'is'),('that', 'i')]
for w in wl:
    print("Bigram: {}\nCount: {}\nPost Add 1 Smoothing Count: {}\nPost Add 1 Smoothing Probability: {}\n".format(w,dict_wordseq_count[2][tuple(w)],add1_bigram_count(w),add1_bigram_prob(w)))

Examples with Drastic change in Counts:

Bigram: ('of', 'the')
Count: 581
Post Add 1 Smoothing Count: 129.71034059527463
Post Add 1 Smoothing Probability: 0.05952746241178276

Bigram: ('it', 'is')
Count: 269
Post Add 1 Smoothing Count: 40.45317220543807
Post Add 1 Smoothing Probability: 0.030211480362537766

Bigram: ('that', 'i')
Count: 185
Post Add 1 Smoothing Count: 27.920805369127518
Post Add 1 Smoothing Probability: 0.02080536912751678



## Probability of Sentence using Bigram with Add-1 smoothing

In [13]:
def probability_sentence_add1_bigram(sent):
    sent=sent_splitter(sent,add_tag=1)
    prob = 0
    for w in sent[:-1]:
        index = sent.index(w)
        if index+2<=len(sent):
            try:
                temp = add1_bigram_prob(sent[index:index+2])
            except KeyError:
                temp=0                
#             print(temp)
            if temp==0: # How to take log----take -inf
                probt = float("-inf")
            else:
                probt = math.log(temp)
            prob += probt
#             print (prob)
    return math.exp(prob)


print("Example Sentences with 2-gram model Probability after Add-1 Smoothing:")
sl = ['i sat down','do you mean that','the lamps had been lit but the blinds had not been drawn','so far i had got when we went to visit the scene of action']
for s in sl:
    print('\nSentence: {}'.format(s))
    print("Probability:{}".format(probability_sentence_add1_bigram(s)))

Example Sentences with 2-gram model Probability after Add-1 Smoothing:

Sentence: i sat down
Probability:5.3189834149362046e-11

Sentence: do you mean that
Probability:5.9503724659307494e-15

Sentence: the lamps had been lit but the blinds had not been drawn
Probability:2.261365020126562e-40

Sentence: so far i had got when we went to visit the scene of action
Probability:1.9439558762275912e-47


# Good Turing Estimation

In [14]:
freq_count_ngram = {1:{},2:{},3:{},4:{}}
dict_ngram_sorted_by_count = {1:{},2:{},3:{},4:{}}
for i in range(1,5):
    dict_ngram_sorted_by_count[i] = sorted(dict_wordseq_count[i].items(), key=lambda value: value[1], reverse=True)

for i in range(1,5):
    for k in dict_ngram_sorted_by_count[i]:
        try:
            freq_count_ngram[i][k[1]]+=1
        except KeyError:
            freq_count_ngram[i].update({k[1]:1})

            
# print(dict_ngram_sorted_by_count[2])
# print(freq_count_ngram[1])

good_turing_counts = {1:{},2:{},3:{},4:{}}
# good_turing_prob = {1:{},2:{},3:{},4:{}}
for i in range(1,5):
    for k in freq_count_ngram[i]:
        try:
            c_new_gt = (k+1)*freq_count_ngram[i][k+1]/(freq_count_ngram[i][k])
            good_turing_counts[i].update({k: c_new_gt})
#             good_turing_prob[i].update({k: c_new_gt/(len(dict_ngram[i])) })
        except KeyError:
            good_turing_counts[i].update({k: 0})
#             good_turing_prob[i].update({k: 0})
    
# print(good_turing_counts)
d_good_turing = {}
for k in range(1,5):
    s = 0
    print("\n{}-Gram Model:\n".format(k))
    for i in range(1,11):
        print("\tCount= {} -> Good Turing Count= {}".format(i,good_turing_counts[k][i]))
        s += i-good_turing_counts[k][i]
    s = s/10
    print("\n Discounting value d for {}-gram= {}\n".format(k,s))
    d_good_turing.update({k:s})  



1-Gram Model:

	Count= 1 -> Good Turing Count= 0.6346782988004362
	Count= 2 -> Good Turing Count= 1.6546391752577319
	Count= 3 -> Good Turing Count= 2.2866043613707165
	Count= 4 -> Good Turing Count= 3.7329700272479562
	Count= 5 -> Good Turing Count= 4.532846715328467
	Count= 6 -> Good Turing Count= 4.8019323671497585
	Count= 7 -> Good Turing Count= 7.154929577464789
	Count= 8 -> Good Turing Count= 7.086614173228346
	Count= 9 -> Good Turing Count= 7.5
	Count= 10 -> Good Turing Count= 9.68

 Discounting value d for 1-gram= 0.5934785304151798


2-Gram Model:

	Count= 1 -> Good Turing Count= 0.32000756072204894
	Count= 2 -> Good Turing Count= 1.1139988186650915
	Count= 3 -> Good Turing Count= 1.9533404029692472
	Count= 4 -> Good Turing Count= 2.8718783930510314
	Count= 5 -> Good Turing Count= 4.378071833648393
	Count= 6 -> Good Turing Count= 4.461139896373057
	Count= 7 -> Good Turing Count= 6.861788617886178
	Count= 8 -> Good Turing Count= 6.5260663507109005
	Count= 9 -> Good Turing Coun

## Good Turing Probabilities for Bigrams using Discounting value

In [18]:
d = d_good_turing[2]
good_turing_count_d_bigram = {}
for k in range(1,11):
    good_turing_count_d_bigram.update({k:good_turing_counts[2][k]})

for k in range(11,max(freq_count_ngram[2].keys())):
    try:
        good_turing_count_d_bigram.update({k:freq_count_ngram[2][k]-d})
    except KeyError:
        good_turing_count_d_bigram.update({k:0})

#         if freq_count_ngram[2][k]>d:
#             prob_gt_bigram.update({k: (freq_count_ngram[2][k]-d)/(len(dict_ngram[2]))})
#         else:
#             prob_gt_bigram.update({k: (freq_count_ngram[2][1])/(len(dict_ngram[2]))})
#     except KeyError:
#         prob_gt_bigram.update({k: (freq_count_ngram[2][1])/(len(dict_ngram[2]))})

good_turing_count_d_bigram

{1: 0.32000756072204894,
 2: 1.1139988186650915,
 3: 1.9533404029692472,
 4: 2.8718783930510314,
 5: 4.378071833648393,
 6: 4.461139896373057,
 7: 6.861788617886178,
 8: 6.5260663507109005,
 9: 8.03921568627451,
 10: 7.780487804878049,
 11: 85.93059953651785,
 12: 59.93059953651785,
 13: 67.93059953651785,
 14: 56.93059953651785,
 15: 44.93059953651785,
 16: 37.93059953651785,
 17: 38.93059953651785,
 18: 32.93059953651785,
 19: 25.93059953651785,
 20: 19.93059953651785,
 21: 17.93059953651785,
 22: 20.93059953651785,
 23: 14.930599536517851,
 24: 16.93059953651785,
 25: 14.930599536517851,
 26: 19.93059953651785,
 27: 15.930599536517851,
 28: 11.930599536517851,
 29: 7.9305995365178505,
 30: 9.930599536517851,
 31: 4.9305995365178505,
 32: 5.9305995365178505,
 33: 9.930599536517851,
 34: 4.9305995365178505,
 35: 2.9305995365178505,
 36: 2.9305995365178505,
 37: 3.9305995365178505,
 38: 3.9305995365178505,
 39: 7.9305995365178505,
 40: 1.9305995365178508,
 41: 0.9305995365178508,
 42: 

# Perplexity

### Pre processing to find Perplexity for Bigram with Good Turing Smoothing

In [ ]:
n=2
list_bigram_testing = []
for w in words_sent_corpus_testing:
    for w2 in list(ngrams(w,n)):
        if n==1:
            list_ngram_testing.append((w2))
        else:
            if w2!=('','','','') and w2!=('','','') and w2!=('','') and w2!=(''):
                list_bigram_testing.append(w2)

                
dict_bigram_count_testing={} #counts of bigrams in testing portion of corpus     
if len(dict_bigram_count_testing)!=0: # to ensure this function isn't executed twice in the notebook
    pass
else:
    for i in list_bigram_testing:
        try:
            dict_bigram_count_testing[i]+=1
        except KeyError:
            dict_bigram_count_testing.update({i:1})

# dict_bigram_count_testing
# list_bigram_testing

## Perplexity for Bigram after Add-1 Smoothing and Good Turing Smoothing

In [ ]:
# finding bigrams in testing
list_unique_bigram_testing=[]
list_words=[]
for sent in words_sent_corpus_testing:
    l_temp=[]
    for i in range(len(sent)-1):
        if (sent[i],sent[i+1]) not in list_unique_bigram_testing:
            list_unique_bigram_testing.append((sent[i],sent[i+1]))
        l_temp.append(sent[i])
    list_words.append(l_temp)

c = 0 # total no of bigrams
c2 = 0
perp = 0
perp_gt = 0
for s in list_words:
    for i in range(len(s)-1):
        c += 1
        p = add1_bigram_prob(tuple([s[i],s[i+1]]))
        try:
            pgt = prob_gt_bigram[dict_bigram_count_testing[tuple([s[i],s[i+1]])]]
        except KeyError:
            continue
        perp += math.log(p)
        perp_gt += math.log(pgt)

# print(perp_t)
# print('len(list_bigrams_testing):',len(list_bigrams_testing))
# print('len(list_words):',len(list_words))
perp = perp*(-1)/c
perp = math.exp(perp)

perp_gt = perp_gt*(-1)/c
perp_gt = math.exp(perp_gt)

print("Perplexity for Bigram with Add-1 smoothing is:{}".format(round(perp,2)))
print("Perplexity for Bigram with Good Turing smoothing is:{}".format(round(perp_gt,2)))